In [18]:
import pandas as pd
import numpy as np

In [19]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [26]:
from sklearn.model_selection import train_test_split

In [35]:
df = pd.read_csv("../datasets/mushrooms.csv")

In [36]:
df.head()

class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

In [37]:
df.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [38]:
df.nunique()

class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64

In [135]:
X = df.copy()
Y = X.pop("class")
cols = []

for i in X:
    col = df[i]
    if col.nunique() > 2:
        print (f"One Hot -> {i}")
        cols.append(pd.get_dummies(col,prefix=i))
    else:
        print (f"Binary -> {i}")
        label,*_ = col.unique()
        col = (col == label).astype(np.uint8)
        cols.append(pd.DataFrame(col))
        
X = pd.concat(cols,axis=1)
Y = (Y == 'p').astype(np.uint8)

One Hot -> cap-shape
One Hot -> cap-surface
One Hot -> cap-color
Binary -> bruises
One Hot -> odor
Binary -> gill-attachment
Binary -> gill-spacing
Binary -> gill-size
One Hot -> gill-color
Binary -> stalk-shape
One Hot -> stalk-root
One Hot -> stalk-surface-above-ring
One Hot -> stalk-surface-below-ring
One Hot -> stalk-color-above-ring
One Hot -> stalk-color-below-ring
Binary -> veil-type
One Hot -> veil-color
One Hot -> ring-number
One Hot -> ring-type
One Hot -> spore-print-color
One Hot -> population
One Hot -> habitat


In [136]:
X,x,Y,y = train_test_split(X.values,Y.values)

In [138]:
inp = Input(shape=(X.shape[1:]),name="input")

d1 = Dense(128,activation="relu")(inp)
d2 = Dense(64,activation="relu")(d1)
d3 = Dense(32,activation="relu")(d2)
d4 = Dense(16,activation="relu")(d3)

out = Dense(1,activation="sigmoid",name="out")(d4)

model = Model([inp],[out])

In [139]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 112)]             0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               14464     
_________________________________________________________________
dense_26 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_27 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_28 (Dense)             (None, 16)                528       
_________________________________________________________________
out (Dense)                  (None, 1)                 17        
Total params: 25,345
Trainable params: 25,345
Non-trainable params: 0
_______________________________________________________

In [140]:
X.shape,Y.shape,x.shape,y.shape

((6093, 112), (6093,), (2031, 112), (2031,))

In [141]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [142]:
model.fit(X,Y,epochs=3,batch_size=8,validation_data=(x,y))

Epoch 1/3
762/762 [==============================] - 3s 4ms/step - loss: 0.0368 - accuracy: 0.9903 - val_loss: 1.6909e-04 - val_accuracy: 1.0000
Epoch 2/3
762/762 [==============================] - 3s 3ms/step - loss: 8.2424e-05 - accuracy: 1.0000 - val_loss: 3.9781e-05 - val_accuracy: 1.0000
Epoch 3/3
762/762 [==============================] - 3s 4ms/step - loss: 2.3725e-05 - accuracy: 1.0000 - val_loss: 1.5350e-05 - val_accuracy: 1.0000


In [147]:
!mkdir "../saved_models/mushroom_classification"

A subdirectory or file ../saved_models/mushroom_classification already exists.


In [148]:
open("../saved_models/mushroom_classification/model.json","w+").write(model.to_json())
model.save_weights("../saved_models/mushroom_classification/model")